In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv (r'./categories_version_1.csv')

df.drop(['Unnamed: 0'], axis=1, inplace=True)

if 'categories_version' in list(df.columns):
    df.drop(['categories_version'], axis=1, inplace=True)

    key_list = ['Art', 'Band', 'Chef', 'Mood', 'Show', 'Actor', 'Brand', 'Cause', 'Color', 'Event', 'Gamer', 'Legal', 'Music', 'Topic', 'Artist', 'Author', 'Course', 'Dancer', 'Editor', 'Sports', 'Writer', 'Athlete', 'Blogger', 'Cuisine', 'Finance', 'Profile', 'Science', 'Comedian', 'Designer', 'Diseases', 'Election', 'Fan Page', 'Language', 'Locality', 'Musician', 'Producer', 'Community', 'Education', 'Orchestra', 'Residence', 'Scientist', 'Surgeries', 'Journalist', 'Agriculture', 'Labor Union', 'Nationality', 'Real Estate', 'Social Club', 'Sports Club', 'TV & Movies', 'Visual Arts', 'Work Status', 'Armed Forces', 'Concert Tour', 'Entrepreneur', 'Just For Fun', 'Meeting Room', 'Talent Agent', 'Ticket Sales', 'Work Project', 'Fashion Model', 'Film Director', 'Fitness Model', 'Literary Arts', 'Local Service', 'Musician/Band', 'Public Toilet', 'Satire/Parody', 'Sports Season', 'Video Creator', 'Work Position', 'Not a Business', 'Campus Building', 'Digital Creator', 'Food & Beverage', 'Harmonized Page', 'Hotel & Lodging', 'Performance Art', 'Performing Arts', 'Sports Promoter', 'Theatrical Play', 'Exchange Program', 'Medical & Health', 'News Personality', 'Spiritual Leader', 'Books & Magazines', 'Community Service', 'Editorial/Opinion', 'Shopping & Retail', 'University (NCES)', 'University Status', 'Media/News Company', 'Outdoor Recreation', 'Youth Organization', 'City Infrastructure', 'Sports & Recreation', 'Arts & Entertainment', 'Charity Organization', 'Motivational Speaker', 'Private Members Club', 'Advertising/Marketing', 'Sorority & Fraternity', 'Nonprofit Organization', 'Religious Organization', 'Theatrical Productions', 'Commercial & Industrial', 'Travel & Transportation', 'Country Club / Clubhouse', 'Media Restoration Service', 'Religious Place of Worship', 'Automotive, Aircraft & Boat', 'Landmark & Historical Place', 'Public & Government Service', 'Automated Teller Machine (ATM)', 'Beauty, Cosmetic & Personal Care', 'Science, Technology & Engineering', 'Non-Governmental Organization (NGO)', 'Environmental Conservation Organization']

    count = 0
    accept_percent = 0.5
    for key in key_list:
        i = df[key].isin([0]).sum()
        if i >= len(df.index) * accept_percent:
            print(f'{key} : {i}')
            df.drop(key, axis=1, inplace=True)
            count +=1
    print(count)

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(50, 50))
heatmap = sns.heatmap(df.corr(), vmin=0, vmax=1, annot=True, xticklabels=True, yticklabels=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=25);

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import numpy as np

In [ ]:
class AccuracyMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        best_class = np.argmax(approxes, axis=0)
        
        accuracy_sum = 0
        weight_sum = 0 

        for i in range(len(target)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            accuracy_sum += w * (best_class[i] == target[i])

        return accuracy_sum, weight_sum

In [ ]:
# feature matrix
X = df.drop(['ans_type'], axis=1, inplace=False)

# target vector
y = df['ans_type'] - 1

# class labels
labels = []
for i in df.drop(['ans_type'], axis=1, inplace=False).columns:
    labels.append(i)

random_state = 3
#brute force
list_pass = []
for random_state in range(1,101):
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state, stratify=y) # 70% training and 30% test
    
    pool_train = Pool(X_train, y_train, feature_names=list(X_train.columns))
    pool_test = Pool(X_test, y_test, feature_names=list(X_test.columns))

    model = CatBoostClassifier(max_depth=15, max_ctr_complexity=15, iterations=1, loss_function='MultiClass', eval_metric=AccuracyMetric(), learning_rate=0.03, random_state=random_state)
    model.fit(pool_train, use_best_model=True, eval_set=pool_test)

    accuracy = max(model.evals_result_.get('validation').get('AccuracyMetric'))
    if accuracy>=0.7:
        print(f'random_state : {random_state}  accuracy : {accuracy*100}%%')
        list_pass.append((random_state,accuracy))


In [ ]:
print(list_pass)

In [ ]:
# feature matrix
X = df.drop(['ans_type'], axis=1, inplace=False)

# target vector
y = df['ans_type'] - 1

# class labels
labels = []
for i in df.drop(['ans_type'], axis=1, inplace=False).columns:
    labels.append(i)

random_state = max(list_pass, key=lambda item:item[1])[0]
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state, stratify=y) # 70% training and 30% test

In [ ]:
pool_train = Pool(X_train, y_train, feature_names=list(X_train.columns))
pool_test = Pool(X_test, y_test, feature_names=list(X_test.columns))

model = CatBoostClassifier(max_depth=15, max_ctr_complexity=15, iterations=1, loss_function='MultiClass', eval_metric=AccuracyMetric(), learning_rate=0.03, random_state=random_state)
model.fit(pool_train, use_best_model=True, eval_set=pool_test)

print("Count of trees in model = {}".format(model.tree_count_))

In [ ]:
model.plot_tree(tree_idx=0, pool=pool_train)

In [ ]:
max(model.evals_result_.get('validation').get('AccuracyMetric'))

In [ ]:
model.evals_result_.get('validation')

In [ ]:
y_pred = model.predict(pool_test)
print(classification_report(y_test,y_pred))

In [ ]:
from graphviz import Source

graph = model.plot_tree(tree_idx=0, pool=pool_train)
png = Source(graph, format="png") 
png
png.render("decision_tree_graphivz_cat")